In [2]:
# %pip install caas_jupyter_tools
%pip install openpyxl
#terminal: python -m pip install --user openpyxl

Note: you may need to restart the kernel to use updated packages.


c:\Users\marin\OneDrive\Desktop\UdeM\Nube\Clase_1_2_3\Clases\MLOps_UdM\.venv\Scripts\python.exe: No module named pip


In [1]:
import openpyxl

In [ ]:
# Reejecutar el análisis heurístico con el nuevo archivo adjunto para proponer etiquetas iniciales
import pandas as pd
from collections import Counter
import re

# from caas_jupyter_tools import display_dataframe_to_user

# Leer el archivo adjunto
df2 = pd.read_excel("Libro3.xlsx", engine="openpyxl")

# Asegurar que la columna exista
col = "OBSERVACION TERRENO"
if col not in df2.columns:
    raise ValueError(f"No se encontró la columna '{col}'. Columnas disponibles: {df2.columns.tolist()}")

# Normalizar texto
texts = df2[col].dropna().astype(str).str.lower().str.replace(r"[^a-z0-9áéíóúñü\s]", " ", regex=True)

# Buckets de palabras clave basados en el contexto de telecomunicaciones
buckets = {
    "Verificación de datos técnicos y soporte puntual": [
        "verific", "soporte", "revis", "prueba", "parametros", "diagnostico", "medicion", "visita", "verificar"
    ],
    "Verificación de plan o servicios contratados": [
        "plan", "paquete", "servicio", "contrato", "velocidad", "plan contratado", "subir", "upgrade"
    ],
    "Problemas de conectividad (señal PON inestable)": [
        "pon", "señal", "inestable", "sin servicio", "intermitencia", "desconect", "no navega", "caida", "sin conexion"
    ],
    "Reemplazo de equipos (ONT/decodificadores)": [
        "ont", "deco", "router", "equipo", "modem", "cambio", "reemplazo", "instalar", "instalacion"
    ],
    "Actualización de infraestructura (cambio de puertos GPON)": [
        "gpon", "puerto", "nodo", "splitter", "fibra", "red", "infraestructura", "migracion", "actualizacion"
    ],
    "Aprovisionamiento de servicios": [
        "aprovision", "provision", "alta", "activar", "subida", "configuracion", "nueva conexion", "creacion"
    ]
}

# Función heurística para asignar etiquetas
def suggest_label(text):
    scores = {k: 0 for k in buckets}
    for k, kws in buckets.items():
        for kw in kws:
            if kw in text:
                scores[k] += 1
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "Otros"

# Aplicar a todas las observaciones
labels_suggested = texts.apply(suggest_label)
label_counts = labels_suggested.value_counts().rename_axis("Etiqueta sugerida").reset_index(name="Frecuencia")

# Mostrar resultados resumidos
display("Propuesta de etiquetas heurísticas (telecomunicaciones)", label_counts.head(20))

# Exportar a Excel
# df.to_excel("Libro3_etiquetado.xlsx", index=False)


'Propuesta de etiquetas heurísticas (telecomunicaciones)'

,Etiqueta sugerida,Frecuencia
0,Reemplazo de equipos (ONT/decodificadores),614
1,Problemas de conectividad (señal PON inestable),298
2,Verificación de plan o servicios contratados,250
3,Aprovisionamiento de servicios,124
4,Actualización de infraestructura (cambio de pu...,123
5,Otros,87
6,Verificación de datos técnicos y soporte puntual,36
